In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [2]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [3]:
from langchain_community.document_loaders import PyPDFLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(dir, filename))
        pages += loader.load()
    break # too much time, only load one file
pages
pages = [pages[0]]

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [5]:
import sys
sys.path.append("..")
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph)
graph_builder.graph_build(pages)
graph_builder.chunk_list

[{'chunk_id': 'fd17b6f6-8ad5-4cd4-b45d-474da1c9684c',
  'chunk_doc': Document(metadata={'source': '../data/台灣人壽美鑫美利美元利率變動型終身壽險.pdf', 'page_number': 1}, page_content='第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及通知】   第十三條   本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三 項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：   一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費 ，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未撤銷者， 應依保單條款 第十六條約定辦理。   二、現金給付：選擇現金給付者')},
 {'chunk_id': 'bc2887fb-e2e9-4ed6-85c2-b18906e4d488',
  'chunk_doc': Document(metadata={'source': '../data/台灣人壽美鑫美利美元利率變動型終身壽險.pdf', 'page_number': 1}, page_content='，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第 六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。   三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依 據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一 併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其 孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方 式給付。   要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知 本公司變更前項給付方式')},
 {'chunk_id': 'd20b6bd7-f3ab-4885-ac0f-ea61aa94cdad',
  'chunk_doc': Document(met

In [6]:

graph_documents = graph_builder.get_graph_from_llm(llm, graph_builder.chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

[GraphDocument(nodes=[Node(id='台灣人壽', type='Organization'), Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), Node(id='增值回饋分享金', type='Financial_incentive'), Node(id='被保險人', type='Person')], relationships=[Relationship(source=Node(id='台灣人壽', type='Organization'), target=Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), type='PROVIDES'), Relationship(source=Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), target=Node(id='增值回饋分享金', type='Financial_incentive'), type='INCLUDES'), Relationship(source=Node(id='增值回饋分享金', type='Financial_incentive'), target=Node(id='被保險人', type='Person'), type='BENEFITS')], source=Document(metadata={'combined_chunk_ids': ['fd17b6f6-8ad5-4cd4-b45d-474da1c9684c']}, page_content='第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及通知】   第十三條   本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三 項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：   一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費 ，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未撤銷者， 應依保單條款 第十六條約定辦理。   二、現金給付：選擇現金給付者')),
 GraphDocume

In [7]:
graph.add_graph_documents(graph_documents)

In [8]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='台灣人壽', type='Organization'), Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), Node(id='增值回饋分享金', type='Financial_incentive'), Node(id='被保險人', type='Person')], relationships=[Relationship(source=Node(id='台灣人壽', type='Organization'), target=Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), type='PROVIDES'), Relationship(source=Node(id='美鑫美利美元利率變動型終身壽險', type='Insurance'), target=Node(id='增值回饋分享金', type='Financial_incentive'), type='INCLUDES'), Relationship(source=Node(id='增值回饋分享金', type='Financial_incentive'), target=Node(id='被保險人', type='Person'), type='BENEFITS')], source=Document(metadata={'combined_chunk_ids': ['fd17b6f6-8ad5-4cd4-b45d-474da1c9684c'], 'id': 'f77aada19339db78284f6ceaa82c26c4'}, page_content='第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及通知】   第十三條   本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三 項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：   一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費 ，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未

In [9]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)